<a href="https://colab.research.google.com/github/mechaturtles/SOST4Website/blob/master/Exercise_%26_Insulin_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise & Insulin Report Functions by Forrest Dipert

**Instructions:** 

Scroll down the page and click the arrow "play button" as you scroll past blocks of code to intialize and run each block. The lower blocks of code must be ran after the earlier blocks as they are dependent on eachother.

**Inputs:**

This function intakes data from a continuous glucose monitor (e.g. Dexcom G6) like Estimated Glucose Value (EGVs), and events (e.g. exercise, carb intake, insulin administration, etc...).

**Outputs:**

The Exercise report delivers the average and standard deviation of EGVs for a period after exercise takes place. Allowing patients and providers aware of any trends in post-exercise hypoglycecmia.

The Insulin report delivers the amount of fast and long acting insulin administered over the time period observed. Better informing patients and providers of the individual's insulin needs over time and when prescription may need to be refilled.

Estimated Glucose Values (egvs) dictionary - used for Functions 1 & 2. Deliverd from the Dexcom G6 glucose monitor

    Keys:
      time = ["hours(24hr)minutes(base 100)", ...]
      realtimeValue1 = [egvs data as floats]
      trend1 = [egvs trendrate data as floats]

      Events dictionary list - t_(time), there is an event for each time point delivered (if no event occurs then the event is None)
        Event types are stored in first position of array (carbs, Exercise, insulin, None)
          [Carbs, grams]
          [Exercise, type (light, medium, hard), hours (float)]
          [insulin, type (fastActing, longActing), units (float)]

In [5]:
egvs = dict(
    #Hour (24 hr), minute (base 100), strings
    time = [ "1600", "1650", "1700", "1750", "1800", "1850", "1900", "1950", "2000", "2050"],
    #Estimated Glucose Value data, test case #1 
    realtimeValue1 = [130.0, 120.0, 110.0, 90.0, 80.0, 70.0, 60.0, 50.0, 40.0, 30.0],
    #an 'event' is stored at each time, if no event occurs then the value is null. Event types are carbs, insulin, and Exercise
    trend = dict(
        flat1 = [ -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0 ]),
    Events = dict(
          t_1600 = ["carbs", 35],
          t_1650 = ["Exercise", "light", 2.0],
          t_1700 = ["insulin", "fastActing", 2.2],
          t_1750 = [None],
          t_1800 = ["Exercise", "medium", 1.0],
          t_1850 = ["insulin", "fastActing", 1.8],
          t_1900 = ["Exercise", "Hard", 0.5],
          t_1950 = ["insulin", "longActing", 17],
          t_2000 = ["carbs", 50],
          t_2050 = ["Exercise", "light", 0.5], 
        ),
    #trend is the rate at which EGV is changing at any one data point
    )

Function 1: 

Exercise Report() = average and standard deviation of egvs data after exercise as examined over the study period, and sorted by type of exercise.

In [7]:
def EndStudyPoint(k, q): #Computes the position of the end point where we recording the egvs
  lst = [None] * len(egvs["time"])
  i=0
  multiplier = 2
  while i <= (len(egvs["time"]) - 1): #turns the string list of time into
    lst[i] = int(egvs["time"][i])
    i = i + 1
  start = lst[k]
  end = start + (multiplier * 100 * q)
  return min(range(len(lst)), key = lambda i: abs(lst[i] - end))


def average(k, q, TestCase): #Computes the average evgs for a period after exercise
    end = EndStudyPoint(k,q)
    total = 0
    start = k
    while k <= end: 
        total = total + egvs["realtimeValue" + TestCase][k]
        k = k + 1
    avg = total / (end - start + 1)
    return avg

def stdev(k, q, TestCase): #Computes the stdev evgs for a period after exercise
    end = EndStudyPoint(k,q)
    total = 0
    total2 = 0
    start = k
    g = k
    while k <= end: 
        total = total + egvs["realtimeValue" + TestCase][k]
        k = k + 1
    avg = total / (end - start + 1)
    while g <= end:
      total2 = total2 + ((egvs["realtimeValue" + TestCase][g] - avg) ** 2 )
      g = g + 1
    S2 = total2 / (end - start + 1)
    S = S2 ** (1/2)
    
    return S
    

def ExerciseReport():
    TestCase = "1"
    light_stat = [0.0, 0.0]
    medium_stat = [0.0, 0.0]
    hard_stat = [0.0, 0.0]
    j = 0
    for i in egvs["time"]:  
      if egvs["Events"]["t_" + i][0] == "Exercise":
          data = egvs["Events"]["t_" + i]
          if data[1] == "light": #Exercise event is light exercise
            if light_stat[0] == 0:
              light_stat[0] = average(j, data[2], TestCase)
            else:
              light_stat[0] = (light_stat[0] + average(j, data[2], TestCase)) / 2.0

            if light_stat[1] == 0:
              light_stat[1] = stdev(j, data[2], TestCase)
            else:
              light_stat[1] = (light_stat[1] + stdev(j, data[2], TestCase)) / 2.0

          elif data[1] =="medium": #Exercise event is medium exercise
            if medium_stat[0] == 0:
              medium_stat[0] = average(j, data[2], TestCase)
            else:
              medium_stat[0] = (medium_stat[0] + average(j, data[2], TestCase)) / 2.0
            
            if medium_stat[1] == 0:
              medium_stat[1] = stdev(j, data[2], TestCase)
            else:
              medium_stat[1] = (medium_stat[1] + stdev(j, data[2], TestCase)) / 2.0
          else: #Exercise event is hard exercise
            if hard_stat[0] == 0:
              hard_stat[0] = average(j, data[2], TestCase)
            else:
              hard_stat[0] = (hard_stat[0] + average(j, data[2], TestCase)) / 2.0

              if hard_stat[1] == 0:
                hard_stat[1] = stdev(j, data[2], TestCase)
              else:
                hard_stat[1] = (hard_stat[1] + stdev(j, data[2], TestCase)) / 2.0
      
      j = j + 1

      
    Output = "The average and standard deviation of the Estimated Glucose Value after exercise is as follows: \n"
    Output_light = "\t light exercise:  %.1f \u00B1 %.2f mg/dL \n" % (light_stat[0], light_stat[1])   
    Output_medium = "\t medium exercise: %.1f \u00B1 %.2f mg/dL \n" % (medium_stat[0], medium_stat[1])
    Output_hard = "\t hard exercise:   %.1f \u00B1 %.2f mg/dL \n" % (hard_stat[0], hard_stat[1])
    Output = Output + Output_light + Output_medium + Output_hard
    return Output
          

Result_ExerciseReport = ExerciseReport()
print(Result_ExerciseReport)

The average and standard deviation of the Estimated Glucose Value after exercise is as follows: 
	 light exercise:  51.1 ± 14.49 mg/dL 
	 medium exercise: 60.0 ± 14.14 mg/dL 
	 hard exercise:   50.0 ± 0.00 mg/dL 



Function 2:

Insulin Report() = reports the units of insulin administered over a given time frame

In [9]:
def InsulinReport(): #uses egvs dictionary to report the sum of insulin administered in the study period.
  FastSum = 0.0
  LongSum = 0.0
  for i in egvs["time"]:
          
      if egvs["Events"]["t_" + i][0] == "insulin":
        typ  = egvs["Events"]["t_" + i][1]
        if typ == "fastActing":
          FastSum = FastSum + egvs["Events"]["t_" + i][2]
        elif typ == "longActing":
          LongSum = LongSum + egvs["Events"]["t_" + i][2]
    
  time = "Over the following time period %s:%02.0f to %s:%02.0f the following amount of insulin was administered" % (egvs["time"][0][:2], (int(egvs["time"][0][2:4], 10) * 60 / 100), egvs["time"][-1][:2], int(egvs["time"][-1][2:4], 10) * 60 / 100)
  result = " \n \t fast-acting insulin: %.1f units \n \t long-acting insulin: %.1f units" % (FastSum, LongSum)
  return (time + result)
  
Result_InsulinReport = InsulinReport()
print(Result_InsulinReport)


Over the following time period 16:00 to 20:30 the following amount of insulin was administered 
 	 fast-acting insulin: 4.0 units 
 	 long-acting insulin: 17.0 units
